In [3]:
import pandas as pd
import networkx as nx
import json
import csv
import requests
from serpapi import GoogleScholarSearch

## Primo punto
1. A partire da nodes.csv, utilizzare la libreria Python SerpAPI per scaricare, per
ciascuno dei 7 autori elencati:
- author_id: ID identificativo del profilo Google Scholar
- cited_by: numero totale di citazioni ricevute
- interests: elenco degli interessi di ciascun autore
Sfruttando la libreria Python Pandas, usare la struttura dati DataFrame per
aggiornare il file originale con apposite colonne e memorizzarlo nella cartella
/data.

In [4]:
nodesDf = pd.read_csv("./nodes.csv")

display(nodesDf)

,Unnamed: 0,name,affiliations,author_id,cited_by,interests
0,0,David La Barbera,Università Degli Studi Di Udine,MvN8wKEAAAAJ,139,"[{'title': 'Computer Science', 'serpapi_link':..."
1,1,Michael Soprano,Postdoctoral Research Fellow at the University...,ocK0qRUAAAAJ,210,"[{'title': 'Computer Science', 'serpapi_link':..."
2,2,Kevin Roitero,University of Udine,1xd52jMAAAAJ,690,"[{'title': 'Artificial Intelligence', 'serpapi..."
3,3,Stefano Mizzaro,Full professor of Computer Science and Informa...,2wvJC6IAAAAJ,4739,"[{'title': 'Information retrieval', 'serpapi_l..."
4,4,Damiano Spina,"School of Computing Technologies, RMIT University",sLzYrNYAAAAJ,2483,"[{'title': 'Information Retrieval', 'serpapi_l..."
5,5,Gianluca Demartini,Associate Professor at the University of Queen...,PCAiILsAAAAJ,5348,"[{'title': 'Information Retrieval', 'serpapi_l..."
6,6,Eddy Maddalena,"Università degli Studi di Udine, Italy",egHDpFkAAAAJ,553,"[{'title': 'Information Retrieval', 'serpapi_l..."


In [3]:
results = []
for index, row in nodesDf.iterrows():
    # Per trovare il json
    params = {
        "api_key": "68a02f21d1aba0067dc456acf08db10a54a40d415213a0ddc8b1bb0ad7ebd15e",
        "engine": "google_scholar_profiles",
        "hl" : "en",
        "mauthors" : row['name'],
        "affilations" : row['affiliations']
    }
    search = GoogleScholarSearch(params)
    result = search.get_dict() #salva i risultati json come dizionario
    results.append(result) #array dei json
    

In [28]:

author_id = []
cited_by = []
interests = []

#item : richiesta API 
for item in results: 
    #info: singolo profilo di ogni richiesta
    for info in item['profiles']:
        #selezione del profilo info corretto 
        for index, row in nodesDf.iterrows():
            if (info['name'] == row['name'] and info['affiliations'] == row['affiliations']):
                author_id.append(info['author_id']) 
                cited_by.append(info['cited_by'])
                interests.append(info['interests']) 
                
filtered_results = pd.DataFrame({
    'author_id': author_id,
    'cited_by': cited_by,
    'interests':interests
    })

nodesDf1 = pd.concat([nodesDf,filtered_results], axis=1)
display(nodesDf1)



,name,affiliations,author_id,cited_by,interests
0,David La Barbera,Università Degli Studi Di Udine,MvN8wKEAAAAJ,139,"[{'title': 'Computer Science', 'serpapi_link':..."
1,Michael Soprano,Postdoctoral Research Fellow at the University...,ocK0qRUAAAAJ,210,"[{'title': 'Computer Science', 'serpapi_link':..."
2,Kevin Roitero,University of Udine,1xd52jMAAAAJ,690,"[{'title': 'Artificial Intelligence', 'serpapi..."
3,Stefano Mizzaro,Full professor of Computer Science and Informa...,2wvJC6IAAAAJ,4739,"[{'title': 'Information retrieval', 'serpapi_l..."
4,Damiano Spina,"School of Computing Technologies, RMIT University",sLzYrNYAAAAJ,2483,"[{'title': 'Information Retrieval', 'serpapi_l..."
5,Gianluca Demartini,Associate Professor at the University of Queen...,PCAiILsAAAAJ,5348,"[{'title': 'Information Retrieval', 'serpapi_l..."
6,Eddy Maddalena,"Università degli Studi di Udine, Italy",egHDpFkAAAAJ,553,"[{'title': 'Information Retrieval', 'serpapi_l..."


### aggiornamento del file originale nodes.csv
aggiunta al file originale nodes.csv : author_id , cited_by, interests 

In [ ]:
nodesDf1.to_csv('nodes.csv')

## Secondo punto
Per ciascuno dei 7 autori, utilizzare il suo ID per accedere al relativo profilo Google
Scholar e scaricare l’elenco dei suoi coautori, sempre via SerpAPI. Con tale elenco
di nomi: 
1. Utilizzare le SerpAPI per cercare su Google Scholar un ricercatore che
corrisponde a tale nome. Per ciascuno, salvare name, affiliations,
author_id, cited_by e interests in un nuovo DataFrame contenente tutte
queste informazioni relative ai coautori dei 7 autori originari.
2. Concatenare il DataFrame con i 7 autori originari e quello dei coautori
generato al punto 2a in un unico DataFrame.
NOTA BENE: è sufficiente effettuare la ricerca dei profili per nome, non
accedere al loro profilo tramite id.
ASSUNZIONE: in questo caso non potete identificare il profilo corretto tramite
il valore di affiliations, quindi assumete che quello corretto sia il primo
ritornato nella lista di authors.
3. Creare un terzo DataFrame con le colonne author1, author2 che
rappresenta le co-authorship. In tale DataFrame, una riga rappresenta un
arco di coauthorship tra due autori.
ESEMPIO: David La Barbera, Michael Soprano è una riga del DataFrame
creato al punto 2c se Michael Soprano è coautore di David La Barbera. La
co-authorship è binaria, non pesata.
A questo punto avrete prodotto due DataFrame:
- Uno per le informazioni relative agli autori (originali + i relativi coautori) e
contenente per ciascuno di essi i valori di name, affiliations, cited_by,
interests. Salvare come nodes.csv nella cartella /data tale DataFrame.
- Uno per le relazioni di co-authorship dai 7 autori principali verso i relativi
coautori con colonne author1, author2. Salvare come edges.csv nella
cartella /data tale DataFrame.


In [5]:
resultsAuthors = []
for index, row in nodesDf.iterrows():
    # Per trovare il json
    params2 = {
        "api_key": "68a02f21d1aba0067dc456acf08db10a54a40d415213a0ddc8b1bb0ad7ebd15e",
        "engine": "google_scholar_author",
        "hl" : "en",
        "author_id" : row['author_id']
    }
    search = GoogleScholarSearch(params2)
    resultAuthor = search.get_dict() #salva i risultati json come dizionario
    resultsAuthors.append(resultAuthor) #array dei json
    
display(resultsAuthors)


ricerca dei coautori e filtraggio dei duplicati

In [14]:
coAuthors_id =[]
for author in resultsAuthors:
    for coAuthor in author['co_authors']:  
        coAuthors_id.append(coAuthor['author_id'])
coAuthors_id = list(set(coAuthors_id)) #set rimuove i duplicati dall'array

display(coAuthors_id)

['ocK0qRUAAAAJ',
 'IUb-xxMAAAAJ',
 'oup2M48AAAAJ',
 'x483lssAAAAJ',
 'EZrMWWMAAAAJ',
 'Yz35RSYAAAAJ',
 'ArV74ZMAAAAJ',
 'egHDpFkAAAAJ',
 'BF3B4KAAAAAJ',
 'crhkrNcAAAAJ',
 'GuteS00AAAAJ',
 'OFlBSEEAAAAJ',
 'lCZblDwAAAAJ',
 'ZZl7bW8AAAAJ',
 'xz1J-xQAAAAJ',
 'LC62bdYAAAAJ',
 '-ExSN_IAAAAJ',
 '9J7YeR0AAAAJ',
 'sLzYrNYAAAAJ',
 'X_95ZxEAAAAJ',
 'JduIcrAAAAAJ',
 'Vf-AkkUAAAAJ',
 'EoGBsiMAAAAJ',
 'AVDkgFIAAAAJ',
 'MvN8wKEAAAAJ',
 'vb6YKF0AAAAJ',
 'NpccCXwAAAAJ',
 'X9BKWWoAAAAJ',
 'HQZd6wsAAAAJ',
 'a-qHaRIAAAAJ',
 'krOJM2sAAAAJ',
 'Oibami4AAAAJ',
 'opFCmpYAAAAJ',
 'WhPw2I0AAAAJ',
 '7J1vJdEAAAAJ',
 '1xd52jMAAAAJ',
 'srKA534AAAAJ',
 'qOLjlqsAAAAJ',
 'U69fiZMAAAAJ',
 'ifU81ikAAAAJ',
 'SRc8aowAAAAJ',
 'HmqvvQkAAAAJ',
 'UTMHOZMAAAAJ',
 '7VIbKlEAAAAJ',
 'wAr9G5sAAAAJ',
 'TMKfR4AAAAAJ',
 'P15-SKkAAAAJ',
 'PCAiILsAAAAJ',
 '1Jo0EmIAAAAJ',
 'm0-0QxgAAAAJ',
 'K5d-OHEAAAAJ',
 'zBYcXUMAAAAJ',
 'UFeON5oAAAAJ',
 'ZxfOSWMAAAAJ',
 'MnwqYB8AAAAJ',
 'ITxRmxIAAAAJ',
 'DL18RZEAAAAJ',
 'Ay6L7NsAAAAJ',
 'VvLUqbkAAAAJ

rimozione dei sette autori originali 

In [24]:
for index, row in nodesDf.iterrows():
    if row['author_id'] in coAuthors_id:
        coAuthors_id.remove(row['author_id'])
#display(coAuthors_id)
print(len(coAuthors_id))

69


In [26]:
resultsCoAuthors = []
for id in coAuthors_id:
    # Per trovare il json
    params2_1 = {
        "api_key": "68a02f21d1aba0067dc456acf08db10a54a40d415213a0ddc8b1bb0ad7ebd15e",
        "engine": "google_scholar_author",
        "hl" : "en",
        "author_id" : id
    }
    search = GoogleScholarSearch(params2_1)
    resultCoAuthor = search.get_dict() #salva i risultati json come dizionario
    resultsCoAuthors.append(resultCoAuthor) #array dei json
    

In [36]:
author_id = []
cited_by = []
interests = []
name = []
affiliations = []

for fileJ in resultsCoAuthors: 
    name.append(fileJ['author']['name'])
    affiliations.append(fileJ['author']['affiliations'])
    author_id.append(fileJ['search_parameters']['author_id']) 
    cited_by.append(fileJ['cited_by']['table'][0]['citations']['all'])
    interests.append(fileJ['author']['interests'][0]) #errore keyerror
                
coAuthorDf = pd.DataFrame({
    'name' : name,
    'affiliations':affiliations,
    'author_id': author_id,
    'cited_by': cited_by,
    'interests':interests
})
display(coAuthorDf)
#nodesDf1 = pd.concat([nodesDf,filtered_results], axis=1)
#display(nodesDf1)



KeyError: 'interests'